In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from typing import List, Tuple
import pandas as pd

In [ ]:
# Custom functions
from some_functions import merge_without_suffixes, normalize_by_production, get_info_for_ids, create_sankey_diagram

In [ ]:
metallican_path = r'C:\Users\mp_ma\OneDrive - polymtlus\Desktop\POST_DOC\Project\canada_metal_sustainability_db'

# Import MetalliCan tables

In [ ]:
main_table = pd.read_csv(metallican_path + r'\database\CSV\main_table.csv')
production_table = pd.read_csv(metallican_path + r'\database\CSV\production_table.csv')
tech_attributes_table = pd.read_csv(metallican_path + r'\database\CSV\tech_attributes_table.csv')
env_table = pd.read_csv(metallican_path + r'\database\CSV\environment_table_new.csv')
technosphere_table = pd.read_csv(metallican_path + r'\database\CSV\technosphere_table.csv')
archetypes_table = pd.read_csv(metallican_path + r'\database\CSV\archetypes_table.csv')
land_table = pd.read_csv(metallican_path + r'\database\CSV\land_occupation_table.csv')

# Create samples based on the best available data

In [ ]:
ids_production = set(production_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_ta = set(tech_attributes_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_environmental = set(env_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_technosphere = set(technosphere_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))
ids_archetypes = set(archetypes_table[['main_id', 'facility_group_id']].apply(tuple, axis=1))

In [ ]:
# Find the intersection of these sets to get common IDs
best_ids = ids_production & ids_ta & ids_environmental & ids_technosphere
second_best_ids = ids_production & ids_ta & ids_environmental
third_best_ids = ids_production & ids_technosphere

In [ ]:
main_best_df = get_info_for_ids(main_table, best_ids)
main_second_best_df = get_info_for_ids(main_table, second_best_ids)
main_third_best_df = get_info_for_ids(main_table, third_best_ids)

In [ ]:
# Export the best and second best IDs to Excel
with pd.ExcelWriter(r'data\MetalliCan\sample_ids.xlsx') as writer:
    main_best_df.to_excel(writer, sheet_name='ta_technosphere_biosphere', index=False)
    main_second_best_df.to_excel(writer, sheet_name='ta_biosphere', index=False)
    main_third_best_df.to_excel(writer, sheet_name='technosphere', index=False)

m# Get the mining method and submethod for the different samples

In [ ]:
main_best_df

In [ ]:
archetypes_best = get_info_for_ids(archetypes_table, best_ids)[['main_id', 'facility_group_id', 'ore_type', 'mining_method', 'mining_submethod']]
archetypes_second_best = get_info_for_ids(archetypes_table, second_best_ids)[['main_id', 'facility_group_id', 'ore_type', 'mining_method', 'mining_submethod']]
archetypes_third_best = get_info_for_ids(archetypes_table, third_best_ids)[['main_id', 'facility_group_id', 'ore_type', 'mining_method', 'mining_submethod']]

In [ ]:
best = merge_without_suffixes(main_best_df, archetypes_best, keys=("main_id", "facility_group_id"), how="left")
second_best = merge_without_suffixes(main_second_best_df, archetypes_second_best, keys=("main_id", "facility_group_id"), how="left")
third_best = merge_without_suffixes(main_third_best_df, archetypes_third_best, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
sankey_best = create_sankey_diagram(best)
sankey_best.write_html(r'data\MetalliCan\sample_best.html', )

In [ ]:
sankey_third = create_sankey_diagram(third_best)
sankey_third.write_html(r'data\MetalliCan\sample_third.html', )

# Create parametrization table for the best IDs

In [ ]:
production_df_best = get_info_for_ids(production_table, best_ids)
production_df_second = get_info_for_ids(production_table,second_best_ids)
production_df_third = get_info_for_ids(production_table,third_best_ids)

In [ ]:
# Remove rows where prod_id do not start with 'PROD' and only keep 'Crude ore' reference point
production_df_best = production_df_best[
    (production_df_best['prod_id'].str.startswith('PROD')) &
    (production_df_best['reference_point'] == 'Crude ore')
]

In [ ]:
production_df_second = production_df_second[
    (production_df_second['prod_id'].str.startswith('PROD')) &
    (production_df_second['reference_point'] == 'Crude ore')
]

In [ ]:
production_df_third = production_df_third[
    (production_df_third['prod_id'].str.startswith('PROD')) &
    (production_df_third['reference_point'] == 'Crude ore')
]

In [ ]:
ta_best = get_info_for_ids(tech_attributes_table, best_ids)[['main_id', 'facility_group_id', 'commodity', 'material_type', 'unit', 'value']]
ta_second_best = get_info_for_ids(tech_attributes_table, second_best_ids)[['main_id', 'facility_group_id', 'commodity', 'material_type', 'unit', 'value']]

In [ ]:
archetypes_best = get_info_for_ids(archetypes_table, best_ids)[['main_id', 'facility_group_id', 'ore_type', 'mining_method', 'mining_submethod']]
archetypes_second_best = get_info_for_ids(archetypes_table, second_best_ids)[['main_id', 'facility_group_id', 'ore_type', 'mining_method', 'mining_submethod']]
archetypes_third_best = get_info_for_ids(archetypes_table, third_best_ids)[['main_id', 'facility_group_id', 'ore_type', 'mining_method', 'mining_submethod']]

In [ ]:
def convert_to_percent(row):
    if row['unit'] == 'g/t':
        return row['value'] / 10000
    elif row['unit'] == '%':
        return row['value']
    else:
        return None


In [ ]:
ta_best['value_%'] = ta_best.apply(convert_to_percent, axis=1)
ta_best.drop(columns=['unit', 'value'], inplace=True)

In [ ]:
ta_second_best['value_%'] = ta_second_best.apply(convert_to_percent, axis=1)
ta_second_best.drop(columns=['unit', 'value'], inplace=True)

In [ ]:
parametrization_best = merge_without_suffixes(ta_best, archetypes_best, keys=("main_id", "facility_group_id"), how="left")
parametrization_second_best = merge_without_suffixes(ta_second_best, archetypes_second_best, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
parametrization_best

In [ ]:
parametrization_second_best

In [ ]:
# To get the reference points available
# production_data_available = production_df.groupby(['main_id', 'facility_group_id'], dropna=False).agg(
#     commodities=('commodity', lambda x: x.unique().tolist()),
#     reference_points=('reference_point', lambda x: x.unique().tolist()),
#     material_types=('material_type', lambda x: x.unique().tolist())
# ).reset_index()


# Clean technosphere table and harmonize units

In [ ]:
technosphere_df_best = get_info_for_ids(technosphere_table, best_ids)
biosphere_df_best = get_info_for_ids(env_table, best_ids)

In [ ]:
technosphere_df_third = get_info_for_ids(technosphere_table, third_best_ids)
biosphere_df_third = get_info_for_ids(env_table, third_best_ids)

In [ ]:
# Separe energy and material
energy_df_best = technosphere_df_best[technosphere_df_best['flow_type'] == 'Energy']
material_df_best = technosphere_df_best[technosphere_df_best['flow_type'] == 'Material use']

In [ ]:
energy_df_third = technosphere_df_third[technosphere_df_third['flow_type'] == 'Energy']
material_df_third = technosphere_df_third[technosphere_df_third['flow_type'] == 'Material use']

In [ ]:
energy_table = technosphere_table[technosphere_table['flow_type'] == 'Energy']

## Deal with energy flows

In [ ]:
## New version

import pandas as pd
import numpy as np

# --- Direct energy units → MJ ---
UNIT_TO_MJ = {
    'mj':   1.0,
    'gj':   1_000.0,
    'tj':   1_000_000.0,
    'j':    1e-6,
    'wh':   0.0036,
    'kwh':  3.6,
    'mwh':  3_600.0,
    'gwh':  3_600_000.0,
}

# --- Volume unit multipliers (to liters) ---
VOLUME_TO_L = {
    'l': 1.0, 'liter': 1.0, 'litre': 1.0, 'liters': 1.0, 'litres': 1.0,
    'kl': 1_000.0, 'kiloliter': 1_000.0, 'kilolitre': 1_000.0,
    'ml': 1_000_000.0, 'megaliter': 1_000_000.0, 'megalitre': 1_000_000.0,
    'gallon': 3.78541, 'gallons': 3.78541,
}

CUBIC_M_TO_M3 = {'m3': 1.0, 'm^3': 1.0, 'cubicmeter': 1.0, 'cubicmeters': 1.0}

# --- Default LHVs (edit with site/company data whenever you can) ---
DEFAULT_LHV = {
    'diesel':      {'MJ/kg': 43.0, 'MJ/L': 38.6, 'density_kg_per_L': 0.835},
    'gasoline':    {'MJ/kg': 44.0, 'MJ/L': 34.2, 'density_kg_per_L': 0.745},
    'heavy_fuel_oil': {'MJ/kg': 40.5, 'MJ/L': 39.69, 'density_kg_per_L': 0.98},
    'coal':        {'MJ/kg': 25.0},
    'natural_gas': {'MJ/m3': 38.0, 'MJ/L': 22.5, 'density_kg_per_L': 0.7},
    'propane':     {'MJ/kg': 46.4, 'MJ/L': 25.3, 'density_kg_per_L': 0.493},
    'electricity': {'MJ/kWh': 3.6},
    'explosives':  {'MJ/kg': 4.0},
    'coke':        {'MJ/kg': 28.0},
    'wood':        {'MJ/kg': 16.0},
}


# --- Subflow canonicalization (aliases + strip pipe suffixes) ---
SUBFLOW_ALIASES = {
    'petrol': 'gasoline',
    'heavy fuel oil': 'heavy_fuel_oil',
    'hfo': 'heavy_fuel_oil',
    'natural gas': 'natural_gas',
    'explosive': 'explosives',
    'lpg': 'propane',
    'surface/underground_emulsion_&_anfo': 'explosives',
    'surface/undergound_emulsion_&_anfo': 'explosives', # to avoid error
}

def _norm_unit(x):
    if pd.isna(x): return None
    return str(x).strip().lower().replace(' ', '')

def _canon_subflow(x):
    if pd.isna(x): return None
    s = str(x).strip().lower()
    if '|' in s:
        s = s.split('|', 1)[0].strip()
    s = SUBFLOW_ALIASES.get(s, s)
    s_us = s.replace(' ', '_')
    return s_us

def standardize_energy_to_MJ(
    df,
    subflow_col='subflow_type',
    unit_col='unit',
    value_col='value',
    lhv_table=None
):
    """
    Convert energy/fuel rows to MJ.
    """
    lhv = (lhv_table or DEFAULT_LHV).copy()
    out = df.copy()

    # Normalize
    out['_unit_n'] = out[unit_col].map(_norm_unit)
    out['_subflow_n'] = out[subflow_col].map(_canon_subflow)
    out[value_col] = pd.to_numeric(out[value_col], errors='coerce')

    # 1) Direct energy units
    direct_mask = out['_unit_n'].isin(UNIT_TO_MJ)
    out.loc[direct_mask, 'value_MJ'] = (
        out.loc[direct_mask, value_col] *
        out.loc[direct_mask, '_unit_n'].map(UNIT_TO_MJ)
    )
    out.loc[direct_mask, 'unit_source'] = 'direct_unit'
    out.loc[direct_mask, 'assumption_note'] = (
        out.loc[direct_mask, '_unit_n'].map(lambda u: f"{u}→MJ factor={UNIT_TO_MJ[u]}")
    )

    # 2) Fuels via LHV
    fuel_rows = ~direct_mask & out['_subflow_n'].notna() & out[value_col].notna()
    for idx in out.index[fuel_rows]:
        sub = out.at[idx, '_subflow_n']
        unit = out.at[idx, '_unit_n']
        val  = out.at[idx, value_col]
        lhv_data = lhv.get(sub)

        if not lhv_data:
            out.at[idx, 'unit_source'] = 'missing_factor'
            out.at[idx, 'assumption_note'] = f"No LHV for subflow={sub}"
            continue

        converted = False

        # A) Mass units (kg, t, lbs)
        if unit in ('kg', 'kilogram', 'kilograms', 't', 'tonne', 'tonnes',
                    'metricton', 'ton', 'lb', 'lbs', 'pound', 'pounds'):
            # Determine multiplier to convert mass unit to kg
            mult_kg = 1.0
            if unit.startswith('t'):
                mult_kg = 1000.0
            elif unit in ('lb', 'lbs', 'pound', 'pounds'):
                mult_kg = 0.453592 # lbs to kg

            factor_kg = lhv_data.get('MJ/kg')
            if factor_kg:
                out.at[idx, 'value_MJ'] = val * mult_kg * factor_kg
                out.at[idx, 'unit_source'] = 'lhv_factor'
                out.at[idx, 'assumption_note'] = f"{sub} MJ/kg={factor_kg} (from {unit})"
                converted = True

        # B) Volume units (L, kL, ML, Gallons)
        elif unit in VOLUME_TO_L:
            mult_L = VOLUME_TO_L[unit]
            factor_l = lhv_data.get('MJ/L')
            if factor_l is None and lhv_data.get('density_kg_per_L') and lhv_data.get('MJ/kg'):
                dens = lhv_data.get('density_kg_per_L')
                factor_kg = lhv_data.get('MJ/kg')
                mass_kg = val * mult_L * dens
                out.at[idx, 'value_MJ'] = mass_kg * factor_kg
                out.at[idx, 'unit_source'] = 'lhv+density'
                out.at[idx, 'assumption_note'] = f"{sub} L→kg via {dens} kg/L; MJ/kg={factor_kg}"
                converted = True
            elif factor_l:
                out.at[idx, 'value_MJ'] = val * mult_L * factor_l
                out.at[idx, 'unit_source'] = 'lhv_factor'
                out.at[idx, 'assumption_note'] = f"{sub} MJ/L={factor_l}"
                converted = True

        # C) Volume units (m3)
        elif unit in CUBIC_M_TO_M3:
            factor_m3 = lhv_data.get('MJ/m3')
            if factor_m3:
                out.at[idx, 'value_MJ'] = val * CUBIC_M_TO_M3[unit] * factor_m3
                out.at[idx, 'unit_source'] = 'lhv_factor'
                out.at[idx, 'assumption_note'] = f"{sub} MJ/m3={factor_m3}"
                converted = True

        if not converted:
            out.at[idx, 'unit_source'] = 'missing_factor'
            out.at[idx, 'assumption_note'] = f"No conversion rule for subflow={sub}, unit={unit}"

    # Final flags
    out['unit_standard'] = np.where(out['value_MJ'].notna(), 'MJ', None)
    out['needs_factor'] = out['value_MJ'].isna() & out[value_col].notna()
    out = out.drop(columns=['_unit_n', '_subflow_n'], errors='ignore')
    return out

In [ ]:
energy_df_best_mj = standardize_energy_to_MJ(energy_df_best,
                                    subflow_col='subflow_type',
                                    unit_col='unit',
                                    value_col='value',
                                    lhv_table=None)   # or pass a custom dict

In [ ]:
energy_df_third_mj = standardize_energy_to_MJ(energy_df_third,
                                    subflow_col='subflow_type',
                                    unit_col='unit',
                                    value_col='value',
                                    lhv_table=None)

## Deal with material flows

In [ ]:
material_table = technosphere_table[technosphere_table['flow_type'] == 'Material use']

In [ ]:
import pandas as pd
import numpy as np

# liters prefixes → L
VOLUME_TO_L = {
    'l': 1.0, 'liter': 1.0, 'litre': 1.0, 'liters': 1.0, 'litres': 1.0,
    'kl': 1_000.0, 'kiloliter': 1_000.0, 'kilolitre': 1_000.0,
    'ml': 1_000_000.0, 'megaliter': 1_000_000.0, 'megalitre': 1_000_000.0,
}

# Default densities (kg/L) – please override with site-specific values when you have them
DEFAULT_DENSITY = {
    # Oils & lubricants family
    'lubricants': 0.88,
    'hydraulic oil': 0.88,
    'transmission oil': 0.88,
    'motor oil': 0.88,
    'drill oil': 0.88,
    'compressor oil': 0.88,

    # Acids (typical commercial concentrations)
    'sulfuric acid (h2so4)': 1.84,    # ~98%
    'hydrochloric acid (hcl)': 1.19,  # ~37%
    'nitric acid (hno3)': 1.51,       # ~68–70%

    # If you have aqueous reagents (e.g., “sodium cyanide solution”) add their conc/density here.
}

# Canonicalize names (left part before '|', lowercased)
ALIASES = {
    'petrol': 'gasoline',
    'grindingmedia': 'grinding media',
    '3/4\'\'balls': 'grinding media',
    '2\'\'balls': 'grinding media',
    '2.5\'\'balls': 'grinding media',
    '5.5\'\'balls': 'grinding media',
    'polyfrothh57': 'polyfroth h57',
    'antiscalant': 'anti-scalant',
}

def _norm_text(x):
    if pd.isna(x): return None
    return str(x).strip()

def _canon_subflow(s):
    if s is None: return None
    # take leftmost token before a pipe and lowercase
    base = s.split('|', 1)[0].strip().lower()
    # strip extra spaces and collapse doubles
    base = ' '.join(base.split())
    return ALIASES.get(base.replace(' ', ''), base)

def standardize_materials_to_t(df, subflow_col='subflow_type', unit_col='unit', value_col='value',
                               density_table=None):
    """
    Convert 'material' rows to tonnes.
    Adds:
      - mass_t : numeric mass in tonnes
      - mass_source : 't','kg→t','L×density→t','missing_density','unknown_unit'
      - mass_note : short note on the assumption used
      - needs_density : True when a volume row had no density mapping
    """
    den = {k.lower(): v for k, v in (density_table or DEFAULT_DENSITY).items()}
    out = df.copy()

    out['_unit_n'] = out[unit_col].astype(str).str.strip().str.lower().str.replace(' ', '', regex=False)
    out['_subflow_n'] = out[subflow_col].map(_canon_subflow)
    out[value_col] = pd.to_numeric(out[value_col], errors='coerce')

    # direct tonnes
    mask_t = out['_unit_n'].isin({'t','tonne','tonnes','metricton','ton'})
    out.loc[mask_t, 'mass_t'] = out.loc[mask_t, value_col].astype(float)
    out.loc[mask_t, 'mass_source'] = 't'
    out.loc[mask_t, 'mass_note'] = 'reported in tonnes'

    # kg → t
    mask_kg = out['_unit_n'].isin({'kg','kilogram','kilograms'})
    out.loc[mask_kg, 'mass_t'] = out.loc[mask_kg, value_col] / 1000.0
    out.loc[mask_kg, 'mass_source'] = 'kg→t'
    out.loc[mask_kg, 'mass_note'] = 'kg/1000'

    # liters family → t using density (kg/L)
    mask_L = out['_unit_n'].isin(VOLUME_TO_L)
    if mask_L.any():
        multL = out.loc[mask_L, '_unit_n'].map(VOLUME_TO_L)
        # find density per row from mapping on canonical subflow
        dens = out.loc[mask_L, '_subflow_n'].map(lambda s: den.get(s if s else '', np.nan))
        mass_t = (out.loc[mask_L, value_col] * multL * dens) / 1000.0
        out.loc[mask_L, 'mass_t'] = mass_t
        out.loc[mask_L, 'mass_source'] = np.where(dens.notna(), 'L×density→t', 'missing_density')
        out.loc[mask_L, 'mass_note'] = np.where(
            dens.notna(),
            (out.loc[mask_L, '_unit_n'].map(str) + f"→L × density kg/L; density=" + dens.map(lambda x: f"{x:g}")),
            "volume reported; no density mapping for this subflow"
        )

    # mark unknown units
    mask_done = mask_t | mask_kg | mask_L
    out.loc[~mask_done & out[value_col].notna(), 'mass_source'] = 'unknown_unit'
    out.loc[~mask_done & out[value_col].notna(), 'mass_note'] = 'no rule for this unit'

    out['needs_density'] = (out['mass_source'] == 'missing_density')

    # clean temp
    out = out.drop(columns=['_unit_n','_subflow_n'])
    return out


In [ ]:
material_df_best_t = standardize_materials_to_t(material_df_best)
material_df_third_t = standardize_materials_to_t(material_df_third)

# Normalize technosphere and biosphere flows by production values

In [ ]:
# Drop rows where needs_factor is True
energy_df_third_mj.drop(energy_df_third_mj[energy_df_third_mj['needs_factor']].index, inplace=True)

In [ ]:
import pandas as pd
def normalize_by_production(df, production_df, value_col='value', prod_col='value_tonnes', prod_agg='sum'):
    df = df.copy()
    # ensure numeric
    df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
    prod = production_df.copy()
    prod[prod_col] = pd.to_numeric(prod[prod_col], errors='coerce')

    # aggregate to unique per key
    main_prod = (prod.dropna(subset=['main_id'])
                    .groupby('main_id', as_index=False)[prod_col]
                    .agg(prod_agg)
                    .rename(columns={prod_col: 'value_tonnes_main'}))
    group_prod = (prod.dropna(subset=['facility_group_id'])
                     .groupby('facility_group_id', as_index=False)[prod_col]
                     .agg(prod_agg)
                     .rename(columns={prod_col: 'value_tonnes_group'}))

    # safe 1:1 merges
    out = df.merge(main_prod, on='main_id', how='left').merge(group_prod, on='facility_group_id', how='left')

    # prefer main_id match, fallback to facility_group_id
    out['value_tonnes_match'] = out['value_tonnes_main'].combine_first(out['value_tonnes_group'])
    out['value_normalized'] = out[value_col] / out['value_tonnes_match']

    # diagnostics
    out['normalization_key'] = None
    out.loc[out['value_tonnes_main'].notna(), 'normalization_key'] = 'main_id'
    out.loc[out['value_tonnes_main'].isna() & out['value_tonnes_group'].notna(), 'normalization_key'] = 'facility_group_id'
    return out

In [ ]:
energy_df_third_mj_norm = normalize_by_production(energy_df_third_mj, production_df_third, value_col='value_MJ')

In [ ]:
material_df_third_norm = normalize_by_production(material_df_third_t, production_df_third, value_col='mass_t', prod_col='value_tonnes', prod_agg='sum')

In [ ]:
material_df_third_norm

In [ ]:
subflow_type_agg = {
    # electricity
    'Electricity consumption|Generated on-site': 'Electricity',
    'Electricity consumption': 'Electricity',
    'Electricity consumption|Grid electricity': 'Electricity',
    'Electricity consumption|Not specified': 'Electricity',
    'Electricity consumption|Non-renewable electricity use': 'Electricity',
    'Solar': 'Electricity',

    # Fuels
    'Diesel': 'Diesel',
    'Diesel|Mobile equipment': 'Diesel',
    'Diesel|Stationary equipment': 'Diesel',
    'Gasoline': 'Gasoline',
    'Gasoline|Mobile equipment': 'Gasoline',
    'Petrol': 'Gasoline',
    'Oil': 'Gasoline',  # usually refers to gasoline
    'Light Fuel & Gasoline': 'Gasoline',
    'Lubricating Oils & Greases': 'Lubricants',
    'Biodiesel': 'Diesel',
    'Propane': 'LPG-Propane',
    'LPG': 'LPG-Propane',
    'LPG|Mobile equipment': 'LPG-Propane',
    'LPG|Stationary equipment': 'LPG-Propane',
    'Acetylene': 'LPG-Propane',
    'Natural gas': 'Natural gas',
    'Naphta': 'Naphtha',  # spelling
    'Aviation fuel': 'Aviation fuel',
    'Non-renewable fuel use': 'Diesel',

    # Explosives
    'Explosives': 'Explosives',
    'Total blasting agents used e.g. anfo': 'Explosives',
    'ANFO': 'Explosives',
    'Emulsion ANFO': 'Explosives',
    'Emulsions': 'Explosives',
    'Emulsion': 'Explosives',
    'Dynamite': 'Explosives',
    'Ammonium nitrate': 'Explosives',  # (treat as energy only if you ANFO-equivalize)

    # Others
    'Used oil': 'Other',   # usually MATERIAL (lubricants); map to energy only if burned
    'Other': 'Other',
}

In [ ]:
# Add a subflow_type_agg column to the energy_std_norm DataFrame based on the dictionnary
energy_df_third_mj_norm['subflow_type_agg'] = energy_df_third_mj_norm['subflow_type'].map(subflow_type_agg).fillna(energy_df_third_mj_norm['subflow_type'])

In [ ]:
# Group by subflow_type_agg and normalization_key, aggregating the normalized values
energy_df_third_mj_norm_agg = (
    energy_df_third_mj_norm
    .groupby(
        [
            'main_id', 'facility_group_id', 'company_id',
            'year', 'flow_type', 'subflow_type_agg'        ],
        dropna=False, as_index=False
    )
    .agg(value_normalized_sum=('value_normalized', 'sum'))
)

# First plots

In [ ]:
energy_plot_df = merge_without_suffixes(energy_df_third_mj_norm_agg, archetypes_third_best, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
energy_plot_df = merge_without_suffixes(energy_plot_df, main_third_best_df, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
energy_plot_df['commodities'].unique()

In [ ]:
material_plot_df = merge_without_suffixes(material_df_third, archetypes_best, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def stripplot_with_barplot(
    data, x, y, hue=None, style=None, log_scale=False,
    outfile=None, dpi=600
):
    """
    Create a scatterplot with overlaid barplot for mean/median.
    Optionally save as high-resolution figure.
    """
    fig, ax = plt.subplots(figsize=(12, 6))

    # Scatterplot (replaces stripplot)
    sns.scatterplot(
        data=data,
        x=x,
        y=y,
        hue=hue,
        style=style,
        ax=ax
    )

    # Barplot for mean/median
    sns.barplot(
        data=data,
        x=x,
        y=y,
        hue=hue,
        estimator='median',
        errorbar=None,
        alpha=0.3,
        width=0.6,
        ax=ax
    )

    # Add vertical separators between x categories
    for i in range(len(data[x].unique()) - 1):
        ax.axvline(i + 0.5, color="grey", linestyle="--", alpha=0.4)

    # Log scale if requested
    if log_scale:
        ax.set_yscale("log")

    # Formatting
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.set_title(
        f"Plot: {y} by {x}" +
        (f" (colored by {hue})" if hue else "") +
        (f" (styled by {style})" if style else "")
    )
    ax.set_xlabel(x)
    ax.set_ylabel(y)

    if hue or style:
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        ax.legend_.remove()

    fig.tight_layout()

    # Save if requested
    if outfile:
        fig.savefig(outfile, dpi=dpi, bbox_inches="tight", format="pdf")
        print(f"Saved figure to {outfile} (dpi={dpi})")

    plt.show()

In [ ]:
stripplot_energy_df = stripplot_with_barplot(
    energy_plot_df,
    "subflow_type_agg",
    "value_normalized_sum",
    hue="mining_method",
    style='commodities',
    log_scale=True,
    outfile="results/stripplot_energy_third_method.pdf",
    dpi=600
)

In [ ]:
stripplot_material_df = stripplot_with_barplot(
    material_plot_df,
    "subflow_type",
    "mass_t",
    hue="mining_submethod",
    log_scale=True,
    outfile="results/stripplot_material_third_submethod.pdf",
    dpi=600
)

# Build relationships for parametrization

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

# ---------------------------
# Core fitting utilities
# ---------------------------

def _safe_loglik(x, rv):
    """Return finite log-likelihood sum for fitted frozen rv."""
    logpdf = rv.logpdf(x)
    logpdf = logpdf[np.isfinite(logpdf)]
    return float(np.sum(logpdf)) if logpdf.size else -np.inf

def _eval_fit(name, rv, x, k):
    """Compute LL, AIC, BIC, KS p for a frozen rv with k free parameters."""
    n = len(x)
    ll = _safe_loglik(x, rv)
    aic = 2 * k - 2 * ll
    bic = k * np.log(n) - 2 * ll
    # KS test using the fitted CDF
    ks_stat, ks_p = stats.kstest(x, rv.cdf)
    return dict(dist=name, ll=ll, aic=aic, bic=bic, ks_p=ks_p, rv=rv, k=k)

def _fit_candidates(x, candidates=("lognormal", "gamma", "weibull_min", "normal")):
    """Fit candidate distributions on strictly positive x."""
    x = np.asarray(x, dtype=float)
    results = []

    # LOGNORMAL: s (sigma), scale=exp(mu), loc fixed 0 => k=2
    if "lognormal" in candidates:
        try:
            s, loc, scale = stats.lognorm.fit(x, floc=0)
            rv = stats.lognorm(s=s, loc=0, scale=scale)
            results.append(_eval_fit("lognormal", rv, x, k=2))
        except Exception:
            pass

    # GAMMA: shape a, scale theta, loc fixed 0 => k=2
    if "gamma" in candidates:
        try:
            a, loc, scale = stats.gamma.fit(x, floc=0)
            rv = stats.gamma(a=a, loc=0, scale=scale)
            results.append(_eval_fit("gamma", rv, x, k=2))
        except Exception:
            pass

    # WEIBULL(min): c (shape), scale, loc fixed 0 => k=2
    if "weibull_min" in candidates:
        try:
            c, loc, scale = stats.weibull_min.fit(x, floc=0)
            rv = stats.weibull_min(c=c, loc=0, scale=scale)
            results.append(_eval_fit("weibull", rv, x, k=2))
        except Exception:
            pass

    # NORMAL: mu, sigma => k=2
    if "normal" in candidates:
        try:
            mu, sigma = stats.norm.fit(x)
            rv = stats.norm(loc=mu, scale=max(sigma, 1e-12))
            results.append(_eval_fit("normal", rv, x, k=2))
        except Exception:
            pass

    # Sort by BIC then AIC
    results.sort(key=lambda d: (d["bic"], d["aic"]))
    return results

def _readable_params(best):
    """Human-readable parameter summary by distribution."""
    rv = best["rv"]
    name = best["dist"]
    # scipy frozen RV stores parameters in args/kwds
    args = getattr(rv, "args", ())
    kw = getattr(rv, "kwds", {})

    if name == "lognormal":
        # s = sigma, scale = exp(mu)
        s = kw.get("s", args[0] if args else None)
        scale = kw.get("scale", args[-1] if args else None)
        mu = np.log(scale) if scale is not None else np.nan
        return dict(mu_log=mu, sigma_log=s, scale=scale)
    elif name == "gamma":
        a = kw.get("a", args[0] if args else None)
        scale = kw.get("scale", args[-1] if args else None)
        return dict(shape=a, scale=scale)
    elif name == "weibull":
        c = kw.get("c", args[0] if args else None)
        scale = kw.get("scale", args[-1] if args else None)
        return dict(shape=c, scale=scale)
    elif name == "normal":
        loc = kw.get("loc", 0.0)
        scale = kw.get("scale", np.nan)
        return dict(mu=loc, sigma=scale)
    return dict()

# ---------------------------
# Public API
# ---------------------------

def fit_parametric_by_group(
    df,
    value_col="value_normalized_sum",
    group_cols=("subflow_type_agg", "mining_method"),
    min_n=6,
    winsorize_pct=None,  # e.g., (0.01, 0.01) for 1% tails
    candidates=("lognormal", "gamma", "weibull_min", "normal"),
    results_csv=None,
    pdf_path=None,
    pdf_log_x=True,
    pdf_log_y=False
):
    """
    Fit parametric distributions to positive data by group.

    Returns
    -------
    results_df : DataFrame with:
        group columns, n, min, max, p05, p50, p95,
        best_dist, AIC, BIC, KS_p, params_json (readable dict)
    Side effects (optional)
    -----------------------
    - results_csv: write CSV
    - pdf_path: write a multi-page PDF with histogram + best-fit PDF
    """
    if isinstance(group_cols, str):
        group_cols = [group_cols]

    # Work copy and basic hygiene
    data = df.copy()
    if value_col not in data.columns:
        raise ValueError(f"Missing column '{value_col}'")
    for gc in group_cols:
        if gc not in data.columns:
            raise ValueError(f"Missing group column '{gc}'")

    # Positive values only
    data = data[np.isfinite(data[value_col]) & (data[value_col] > 0)].copy()

    # Optional winsorization (by group)
    if winsorize_pct is not None:
        low_p, high_p = winsorize_pct
        def _wins_g(g):
            lo = g[value_col].quantile(low_p) if low_p else g[value_col].min()
            hi = g[value_col].quantile(1 - high_p) if high_p else g[value_col].max()
            g[value_col] = g[value_col].clip(lo, hi)
            return g
        data = data.groupby(list(group_cols), group_keys=False).apply(_wins_g)

    records = []
    plot_payload = []  # (group_key, x, best_fit)

    for gkey, g in data.groupby(list(group_cols)):
        x = g[value_col].dropna().values
        n = len(x)

        # Summary stats for LCA Algebraic bounds
        if n == 0:
            rec = {gc: gv for gc, gv in zip(group_cols, gkey)}
            rec.update(dict(n=0, min=np.nan, max=np.nan, p05=np.nan, p50=np.nan, p95=np.nan,
                            best_dist="no_data", AIC=np.nan, BIC=np.nan, KS_p=np.nan, params_json={}))
            records.append(rec)
            continue

        p05, p50, p95 = np.percentile(x, [5, 50, 95])
        xmin, xmax = float(np.min(x)), float(np.max(x))

        if n < min_n:
            rec = {gc: gv for gc, gv in zip(group_cols, gkey)}
            rec.update(dict(n=n, min=xmin, max=xmax, p05=p05, p50=p50, p95=p95,
                            best_dist="insufficient_data", AIC=np.nan, BIC=np.nan, KS_p=np.nan, params_json={}))
            records.append(rec)
            continue

        fits = _fit_candidates(x, candidates=candidates)
        if not fits:
            rec = {gc: gv for gc, gv in zip(group_cols, gkey)}
            rec.update(dict(n=n, min=xmin, max=xmax, p05=p05, p50=p50, p95=p95,
                            best_dist="fit_failed", AIC=np.nan, BIC=np.nan, KS_p=np.nan, params_json={}))
            records.append(rec)
            continue

        best = fits[0]
        params_dict = _readable_params(best)

        rec = {gc: gv for gc, gv in zip(group_cols, gkey)}
        rec.update(dict(
            n=n, min=xmin, max=xmax, p05=p05, p50=p50, p95=p95,
            best_dist=best["dist"], AIC=best["aic"], BIC=best["bic"], KS_p=best["ks_p"],
            params_json=params_dict
        ))
        records.append(rec)
        plot_payload.append((gkey, x, best))

    results_df = pd.DataFrame.from_records(records).sort_values(list(group_cols) + ["best_dist"])

    if results_csv:
        Path(results_csv).parent.mkdir(parents=True, exist_ok=True)
        results_df.to_csv(results_csv, index=False)

    # Multipage PDF (one figure per page; journal-friendly)
    if pdf_path:
        Path(pdf_path).parent.mkdir(parents=True, exist_ok=True)
        with PdfPages(pdf_path) as pdf:
            for gkey, x, best in plot_payload:
                fig, ax = plt.subplots(figsize=(7, 5))
                # Histogram
                n_bins = max(8, int(np.sqrt(len(x))))
                ax.hist(x, bins=n_bins, density=True)

                # Best-fit PDF
                rv = best["rv"]
                xmin = max(1e-12, np.min(x) * 0.8)
                xmax = np.max(x) * 1.2
                xs = np.linspace(xmin, xmax, 400)
                ax.plot(xs, rv.pdf(xs))

                if pdf_log_x:
                    ax.set_xscale("log")
                if pdf_log_y:
                    ax.set_yscale("log")

                title = " | ".join([f"{gc}={gv}" for gc, gv in zip(group_cols, gkey)])
                ax.set_title(f"{title}\nBest: {best['dist'].upper()}  (n={len(x)}, BIC={best['bic']:.1f})")
                ax.set_xlabel(value_col)
                ax.set_ylabel("Density")
                fig.tight_layout()
                pdf.savefig(fig, dpi=600)  # 600 dpi pages
                plt.close(fig)

    return results_df


In [ ]:
results = fit_parametric_by_group(
    df=energy_plot_df,
    value_col="value_normalized_sum",
    group_cols=("subflow_type_agg", "mining_method"),
    min_n=3,  # minimum number of samples per group
    winsorize_pct=(0.01, 0.01),  # set None to disable
    candidates=("lognormal", "gamma", "weibull_min", "normal"),
    results_csv="results/best_fit_distributions.csv",
    pdf_path="results/subflow_archetype_fit_plots.pdf",
    pdf_log_x=True,
    pdf_log_y=False
)

In [ ]:
results

# Grade parametrization

In [ ]:
au_ids = [
'ON-MAIN-687b8c8d',	#Island
'ON-MAIN-7607a50e',	#Young-Davidson
'QC-MAIN-f9e41c2a',	#Lamaque
'GRP-0d911886', #Porcupine #3 sites
'GRP-147b3123', #Timmins Operation #2 sites # Also silver in production
'GRP-14bfbb82', #Seabee Gold Operation #2 sites
'QC-MAIN-9de9bb0d',	#Kiena
'ON-MAIN-c5fefb01',	#Mishi

# Additional ones, only gold in production_df but not in NRCan
'BC-MAIN-8eb8be0d', #Red Chris
'ON-MAIN-0aadf28f', #Rainy River
'ON-MAIN-538513cd', # Hoyle Pond, Part of Porcupine Complex
'ON-MAIN-fefeaee4', # Musselwhite
'QC-MAIN-02884fb5' #Westwood-Doyon
]

## Prepare data

### Production

In [ ]:
# Get the production where either main_id or facility_group_id matches the au_ids qnd reference point is Crude ore
production_au = production_df[
    (production_df['main_id'].isin(au_ids) | production_df['facility_group_id'].isin(au_ids)) &
    (production_df['reference_point'] == 'Crude ore') &
    (production_df['prod_id'].str.startswith('PROD'))
].copy()

In [ ]:
production_au

### Energy

In [ ]:
energy_au = technosphere_table[
    (technosphere_table['main_id'].isin(au_ids) | technosphere_table['facility_group_id'].isin(au_ids)) &
    (technosphere_table['flow_type'] == 'Energy')
].copy()

In [ ]:
energy_au_mj = standardize_energy_to_MJ(energy_au,
                                    subflow_col='subflow_type',
                                    unit_col='unit',
                                    value_col='value',
                                    lhv_table=None)

In [ ]:
energy_au_mj.drop(energy_au_mj[energy_au_mj['needs_factor']].index, inplace=True)


In [ ]:
energy_au_mj_norm = normalize_by_production(energy_au_mj, production_au, value_col='value_MJ')


In [ ]:
energy_au_mj_norm

In [ ]:
# Add a subflow_type_agg column to the energy_std_norm DataFrame based on the dictionnary
energy_au_mj_norm['subflow_type_agg'] = energy_au_mj_norm['subflow_type'].map(subflow_type_agg).fillna(
    energy_au_mj_norm['subflow_type'])
# Group by subflow_type_agg and normalization_key, aggregating the normalized values
energy_au_mj_norm_agg = (
    energy_au_mj_norm
    .groupby(
        [
            'main_id', 'facility_group_id', 'company_id',
            'year', 'flow_type', 'subflow_type_agg'],
        dropna=False, as_index=False
    )
    .agg(value_normalized_sum=('value_normalized', 'sum'))
)

### Biosphere

In [ ]:
biosphere_au = env_table[
    env_table['main_id'].isin(au_ids) | env_table['facility_group_id'].isin(au_ids)
].copy()

In [ ]:
biosphere_au

In [ ]:
biosphere_au.to_csv(r'biosphere_au.csv', index=False)

### Parametrization

In [ ]:
ta_au = tech_attributes_table[
    tech_attributes_table['main_id'].isin(au_ids) | tech_attributes_table['facility_group_id'].isin(au_ids)
].copy()

In [ ]:
ta_au

In [ ]:
ta_au

In [ ]:
ta_au['value_%'] = ta_au.apply(convert_to_percent, axis=1)
ta_au.drop(columns=['unit', 'value'], inplace=True)

In [ ]:
archetypes_au = archetypes_table[
    (archetypes_table['main_id'].isin(au_ids) | archetypes_table['facility_group_id'].isin(au_ids))
].copy()

In [ ]:
parametrization_au = merge_without_suffixes(ta_au, archetypes_au, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
parametrization_au

In [ ]:
energy_au_mj_norm.to_csv("results/energy_au_mj_norm.csv", index=False)
parametrization_au.to_csv("results/parametrization_au.csv", index=False)

In [ ]:
energy_df_mj_norm_agg

In [ ]:
parametrization_au

## Plot data

In [ ]:
energy_grade = merge_without_suffixes(energy_df_mj_norm_agg, parametrization_au, keys=("main_id", "facility_group_id"), how="left")

In [ ]:
energy_grade

In [ ]:
stripplot_energy_grade_df = stripplot_with_barplot(
    energy_grade,
    "value_%",
    "value_normalized_sum",
    hue="mining_method",
    log_scale=False,
    outfile="results/stripplot_energy_grade.pdf",
    dpi=600
)